In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain, JLD
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
first(df2, 5)

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real<lower=0> h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=0,upper=250> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

JLD.save("m4.1s.jld",
  "range", chn.range,
  "a3d", chn.value,
  "names", chn.names,
  "chains", chn.chains)

d = JLD.load(joinpath(ProjDir, "m4.1s.jld"))

chn2 = MCMCChain.Chains(d["a3d"], names=d["names"])
describe(chn2)


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.035, 0.031, 0.032, 0.033) seconds, 0.13 seconds total
Sampling took (0.052, 0.047, 0.047, 0.052) seconds, 0.20 seconds total

                Mean     MCSE  StdDev    5%   50%   95%    N_Eff  N_Eff/s    R_hat
lp__            -892  2.4e-02    1.00  -894  -891  -891  1.7e+03  8.6e+03  1.0e+00
accept_stat__   0.92  1.5e-03   0.099  0.72  0.96   1.0  4.5e+03  2.2e+04  1.0e+00
stepsize__      0.80  5.3e-02   0.074  0.72  0.85  0.89  2.0e+00  1.0e+01  5.5e+13
treedepth__      2.0  1.2e-01    0.60   1.0   2.0   3.0  2.6e+01  1.3e+02  1.1e+00
n_leapfrog__     4.1  2.2e-01     2.4   1.0   3.0   7.0  1.2e+02  6.2e+02  1.0e+00
divergent__     0.00      nan    0.00  0.00  0.00  0.00      nan      nan      nan
energy__         893  3.4e-02     1.4   891   892   895  1.7e+03  8.4e+03  1.0e+00
sigma            7.8  5.7e-03    0.30   7.3   7.

end of m4.1s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*